In [3]:
# USAGE
# python detect_mask_image.py --image examples/example_01.png

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os



In [8]:
# Set the paths for the face detector model and mask detector model
face_detector_path = r"C:\HOPE\Deep Learning\Face Mask Dectection\face_detector"
mask_detector_model_path = r"C:\HOPE\Deep Learning\Face Mask Dectection\mask_detector.h5"

# Input the image path directly here
image_path = r"C:\HOPE\Deep Learning\Face Mask Dectection\images (2).jpeg"

# Load the face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.join(face_detector_path, "deploy.prototxt")
weightsPath = os.path.join(face_detector_path, "res10_300x300_ssd_iter_140000.caffemodel")
net = cv2.dnn.readNet(prototxtPath, weightsPath)


[INFO] loading face detector model...


In [9]:
# Load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
model = load_model(mask_detector_model_path)

# Load the input image from disk
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print(f"[ERROR] Could not read image from path: {image_path}")
    exit()

# Clone the image and grab the image spatial dimensions
orig = image.copy()
(h, w) = image.shape[:2]

# Construct a blob from the image
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))


[INFO] loading face mask detector model...


In [10]:
# Pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
net.setInput(blob)
detections = net.forward()


[INFO] computing face detections...


In [11]:
# Loop over the detections
for i in range(0, detections.shape[2]):
    # Extract the confidence (i.e., probability) associated with the detection
    confidence = detections[0, 0, i, 2]

    # Filter out weak detections by ensuring the confidence is greater than the minimum confidence
    if confidence > 0.5:
        # Compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # Ensure the bounding boxes fall within the dimensions of the frame
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX)), (min(h - 1, endY))

        # Extract the face ROI, convert it from BGR to RGB channel ordering, resize it to 224x224, and preprocess it
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # Pass the face through the model to determine if the face has a mask or not
        (mask, withoutMask) = model.predict(face)[0]

        # Determine the class label and color we'll use to draw the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # Include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # Display the label and bounding box rectangle on the output frame
        cv2.putText(orig, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(orig, (startX, startY), (endX, endY), color, 2)

# Show the output image
cv2.imshow("Output", orig)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
